In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
import pickle

In [4]:
df = pd.read_csv("Churn_Modelling.csv")


In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [7]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
##endcode

label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [12]:
##One Hot Encoding for categorical variables
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, drop='first')
encoded_features = ohe.fit_transform(df[['Geography']])
encoded_df = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(['Geography']))
df = pd.concat([df.drop('Geography', axis=1), encoded_df], axis=1)

In [13]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0


In [14]:
## save the encoders and the scaler
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

with open('ohe.pkl', 'wb') as f:
    pickle.dump(ohe, f)

In [16]:
#divide the data into features and target variable
X = df.drop('Exited', axis=1)
y = df['Exited']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   

## Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime




In [ ]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ##first hidden layer
    Dense(32, activation='relu'), ##second hidden layer 
    Dense(1, activation='sigmoid') ##output layer
])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                768       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
opt = keras.optimizers.Adam(learning_rate=0.001)


In [23]:
## compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
## setup TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [25]:
## Setup Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [26]:
#Train the model
history = model.fit(X_train, y_train, 
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/100
200/200 [==============================] - 0s 1ms/step - loss: 0.4686 - accuracy: 0.7959 - val_loss: 0.3995 - val_accuracy: 0.8288
Epoch 2/100
200/200 [==============================] - 0s 642us/step - loss: 0.3946 - accuracy: 0.8361 - val_loss: 0.3745 - val_accuracy: 0.8462
Epoch 3/100
200/200 [==============================] - 0s 640us/step - loss: 0.3618 - accuracy: 0.8533 - val_loss: 0.3577 - val_accuracy: 0.8494
Epoch 4/100
200/200 [==============================] - 0s 681us/step - loss: 0.3487 - accuracy: 0.8581 - val_loss: 0.3538 - val_accuracy: 0.8487
Epoch 5/100
200/200 [==============================] - 0s 711us/step - loss: 0.3426 - accuracy: 0.8605 - val_loss: 0.3566 - val_accuracy: 0.8469
Epoch 6/100
200/200 [==============================] - 0s 877us/step - loss: 0.3379 - accuracy: 0.8619 - val_loss: 0.3470 - val_accuracy: 0.8537
Epoch 7/100
200/200 [==============================] - 0s 804us/step - loss: 0.3323 - accuracy: 0.8650 - val_loss: 0.3443 - val_accu

In [27]:
model.save('model.h5')

/Users/gaurishmaheshwari/Documents/Churn Project/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
## Load TensorBoard extension in Jupyter Notebook
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [34]:

%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 17248), started 0:00:06 ago. (Use '!kill 17248' to kill it.)